In [1]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import configparser
from scipy import stats

In [2]:
# Read Config file
config = configparser.ConfigParser()
config.read('config.ini')
conn_str = 'Driver={SQL Server};Server=' + config['SQL_DATABASE']['SERVER'] + ';Database=' + config['SQL_DATABASE'][
    'DATABASE'] + ';Trusted_Connection=yes;'

KeyError: 'SQL_DATABASE'

In [ ]:
# DB Connection
conn = pyodbc.connect(conn_str)

In [ ]:
# Read Data
queryDF =pd.read_sql_query(config['SQL_QUERY']['QUERY'], conn)
print(queryDF.head())

In [ ]:
queryDF = queryDF.drop(['order_id','page_id'],axis=1)
print(queryDF.describe(include='all'))

In [ ]:
# Check missing values
queryDF.isnull().sum()

In [ ]:
# product, site_version and title columns have only 5, 2 and 3 unique values, so it is a good opportunity to change them to the category type
for col in ['product', 'site_version', 'title']:
    queryDF[col] = queryDF[col].astype('category')
print(queryDF.info(memory_usage = 'deep'))
queryDF.head()

In [ ]:
print('Products on banners: ', queryDF['product'].unique())

In [ ]:
print('Site versions: ', queryDF.site_version.unique())

In [ ]:
print('Page events: ', queryDF.title.unique())

In [ ]:
# Data Analysis

ax=queryDF.groupby(['site_version']).count()['target'].plot.pie(figsize=(7,7),autopct='%1.0f%%')
ax.set_ylabel('')
ax.set_title('Response distributions for different sites')
plt.show()

In [ ]:
ax=queryDF.groupby(['title']).count()['target'].plot.pie(figsize=(7,7),autopct='%1.0f%%')
ax.set_ylabel('')
ax.set_title('Response distributions for different titles')
plt.show()

In [ ]:
ax=queryDF.groupby(['site_version','title']).count()['target'].unstack('title').plot(kind='bar',figsize=(7,7),grid=True)
ax.set_ylabel('count')
ax.set_title('Breakdowns of titles across different sites')
plt.show()

In [ ]:
ax=queryDF.groupby(['product','site_version']).count()['target'].unstack('site_version').iloc[::-1].plot(kind='barh',figsize=(12,15),grid=True)
ax.set_ylabel('product')
ax.set_xlabel('count')
ax.set_title('Overall distributions of product for different sites')
plt.show()